# Final project

Philip Wollsén Ervius \
phao21@student.bth.se

Amin Afzali \
moaf@student.bth.se

### Libraries

In [12]:
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss, Linear
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.datasets import CIFAR10

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, recall_score, f1_score

from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline



### Preparations
The CIFAR-100 dataset will be transformed to fit the ImageNet-1k data it was pretrained on.

In [13]:
# Preprocessing for CIFAR-100 (resize and normalize to match ImageNet-1K pretraining)

preprocess = transforms.Compose([
    # transforms.Resize(256),       # Resize to 256px on the shorter side
    # transforms.CenterCrop(224),   # Center crop to 224x224
    transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    ),
])


### Download the CIFAR-100 dataset

In [ ]:
# Load CIFAR-100 dataset for evaluation
train_dataset = CIFAR10(root="./data", train=True, transform=preprocess, download=True)
test_dataset = CIFAR10(root="./data", train=False, transform=preprocess, download=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


Using downloaded and verified file: ./data\cifar-10-python.tar.gz
Extracting ./data\cifar-10-python.tar.gz to ./data


### Load the pretrained model

In [9]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x0_5", pretrained=True)

Using cache found in C:\Users\pwoll/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/mobilenetv2/cifar10_mobilenetv2_x0_5-ca14ced9.pt" to C:\Users\pwoll/.cache\torch\hub\checkpoints\cifar10_mobilenetv2_x0_5-ca14ced9.pt
100%|██████████| 2.85M/2.85M [00:01<00:00, 1.61MB/s]


### Define a function to evaluate the model

In [ ]:



all_labels = []
all_predictions = []

i = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.append(labels)
        all_predictions.append(predicted)

        if i > 100:
            break


# Concatenate all tensors
all_labels = torch.cat(all_labels).numpy()
all_predictions = torch.cat(all_predictions).numpy()

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='macro')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


KeyboardInterrupt: 

In [ ]:

# Evaluate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

criterion = CrossEntropyLoss()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    test_loss += loss.item()

    _, predicted = torch.max(outputs, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    accuracy_score(outputs, labels)
    f1_score(outputs, labels)
    recall_score(outputs, labels)

print(f"Test Loss: {test_loss / len(test_loader):.4f}")
print(f"Test Accuracy: {accuracy:.2f}%")


In [32]:
def train(img, label):
    """Trains the model on an image.

    Parameters:
        - img: tensor of preprocessed image.
        - label: int target label for the image.
    """

    if len(img.shape) == 3:
        img = img.unsqueeze(0)

    # Define loss function and optimizer
    criterion = CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

    # Set the model to training mode
    model.train()
    running_loss = 0.0

    # Prepare the input and label
    labels = torch.tensor([label])

    optimizer.zero_grad()
    outputs = model(img)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    # print(f'Loss: {running_loss:.3f}')



## The first attack